In [1]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import FastText 
from tqdm import tqdm

#Impotamos los csvs

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1slJiG-M_oawvkzaqWzZkSmVxxZor6mKQ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_features.csv')

id='1TmL2AY_yymiV8zXVvmbqXsIZoexR9sLs'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_features.csv')


id='1J0GcaPrz3SDrfuRjJiSGapHjCTCy094x'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_limpio.csv')

id='1r9I2BE6oMDpSlQPS9OJIK_d0OW4G6yAG'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_limpio.csv')

id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [4]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')
test_limpio = pd.read_csv('test_limpio.csv')
train_limpio = pd.read_csv('train_limpio.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train['text'] = train_limpio['text']
test['text'] = test_limpio['text']

In [6]:
train['text'] = train['text'].fillna(' ')
test['text'] = test['text'].fillna(' ')
train['keyword'] = train['keyword'].fillna(' ')
test['keyword'] = test['keyword'].fillna(' ')

#Encoding con FastText


In [7]:
id = '1eWJLOjsG04oOtYXJ6eta7VnudJPIlBh_'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('wiki-news-300d-1M-subword.vec')

In [9]:
def load_fasttext():
    
    print('loading word embeddings...')
    embeddings_index = {}
    f = open('wiki-news-300d-1M-subword.vec',encoding='utf-8')
    for line in tqdm(f):
        values = line.strip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('found %s word vectors' % len(embeddings_index))
    
    return embeddings_index

embeddings_index = load_fasttext()

2398it [00:00, 11862.38it/s]

loading word embeddings...


999995it [01:25, 11729.93it/s]


found 999995 word vectors


In [10]:
def get_average(text):
  counter = np.zeros(300)
  words = text.split(" ")
  for word in words:
    if word in embeddings_index.keys():
      counter += embeddings_index[word]
  return counter/len(words)

In [11]:
train['average_words'] = train['text'].apply(lambda x: get_average(x))
test['average_words'] = test['text'].apply(lambda x: get_average(x))

In [27]:
train['encoded_keyword'] = train['keyword'].apply(lambda x: np.mean(get_average(x)))
test['encoded_keyword'] = test['keyword'].apply(lambda x: np.mean(get_average(x)))

#Matriz solamente con el texto codificado

In [13]:
matrix_ft_text_train = pd.DataFrame(np.matrix(list(train['average_words'])))
matrix_ft_text_test = pd.DataFrame(np.matrix(list(test['average_words'])))

In [14]:
matrix_ft_text_train.to_csv('matriz_train_ft_solo_text.csv', index=False)
files.download('matriz_train_ft_solo_text.csv')

In [15]:
matrix_ft_text_test.to_csv('matriz_test_ft_solo_text.csv', index=False)
files.download('matriz_test_ft_solo_text.csv')

#Matriz con el texto y keyword

In [31]:
matrix_ft_keyword_train = matrix_ft_text_train.copy()
matrix_ft_keyword_train[300] = train['encoded_keyword']
matrix_ft_keyword_test = matrix_ft_text_test.copy()
matrix_ft_keyword_test[300] = test['encoded_keyword']

In [32]:
matrix_ft_keyword_train.to_csv('matriz_train_ft_text_keyword.csv', index=False)
files.download('matriz_train_ft_text_keyword.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
matrix_ft_keyword_test.to_csv('matriz_test_ft_text_keyword.csv', index=False)
files.download('matriz_test_ft_text_keyword.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Matriz con texto y keyword codificados + features numericos

In [34]:
numeric_features_test = test_features[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]
numeric_features_train = train_features[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]

In [35]:
ft_features_numericos_train = pd.concat([matrix_ft_keyword_train, numeric_features_train], axis=1, sort=False)
ft_features_numericos_test = pd.concat([matrix_ft_keyword_test, numeric_features_test], axis=1, sort=False)

In [38]:
ft_features_numericos_train.to_csv('ft_features_numericos_train.csv', index=False)
ft_features_numericos_test.to_csv('ft_features_numericos_test.csv', index=False)

In [39]:
files.download('ft_features_numericos_train.csv')
files.download('ft_features_numericos_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>